<img src="imgs/GeorgiaTech_RGB.png" alt="GeorgiaTech_RGB" width="200"/>

# <span style='color:#B3A369'> <b>Generate Basis Models</b> </span>

> <b> Notebook Author:</b> Brian Keith (bkeith9@gatech.edu) <br>
> <b> Purpose:</b> This notebook takes the epoch datasets created and created basis models for each of the four model types to be tested using each dataset. This data is then offloaded for use in later notebooks to test and determine the best dataset for each model. <br>
> 
> **Data Citation:**
> - Andreas Miltiadous and Katerina D. Tzimourta and Theodora Afrantou and Panagiotis Ioannidis and Nikolaos Grigoriadis and Dimitrios G. Tsalikakis and Pantelis Angelidis and Markos G. Tsipouras and Evripidis Glavas and Nikolaos Giannakeas and Alexandros T. Tzallas (2023). A dataset of EEG recordings from: Alzheimer's disease, Frontotemporal dementia and Healthy subjects. OpenNeuro. [Dataset] doi: doi:10.18112/openneuro.ds004504.v1.0.6
>
> **Data Source:**
> - Dataset download as a zip file from: https://nemar.org/dataexplorer/detail?dataset_id=ds004504
>
> **Data Information:**
> - Cognitive and neuropsychological state was evaluated by the international Mini-Mental State Examination (**MMSE**)
>   - The MMSE score ranges from 0 to 30, with *a lower MMSE indicating more severe cognitive decline*.
> - Recordings include the EEG signal from 19 scalp electrodes `Fp1`, `Fp2`, `F7`, `F3`, `Fz`, `F4`, `F8`, `T3`, `C3`, `Cz`, `C4`, `T4`, `T5`, `P3`, `Pz`, `P4`, `T6`, `O1`, & `O2`, and 2 reference electrodes, placed according to the 10–20 international system (`A1` & `A2`).
>   - Sampling Rate = 500 Hz
>   - Resolution = 10 uV/mm
>   - Each recording lasted approximately 13.5 min for the AD group (min = 5.1, max = 21.3), 12 min for the FTD group (min = 7.9, max = 16.9), and 13.8 min for the CN group (min = 12.5, max = 16.5). 
>       - In total, 485.5 min of AD, 276.5 min of FTD, and 402 min of CN recordings were collected and are included in the dataset.
> - 88 participants total
>   - Alzheimer’s disease (**AD**) patients (n=36)
>       - `Group = A`
>   - Frontotemporal dementia (**FTD**) patients (n=23)
>       - `Group = F`
>   - Healthy controls (**CN**) participants (n=29)
>       - `Group = C`
>
> 


# <span style='color:#B3A369'> Initial Tasks </span>

## <span style='color:#003057'> Set Params & Load Necessary Libraries </span>

In [1]:

SEED_VAL = 903027850 #GTID
PALETTE = [
    '#66829a',
    '#003057',
    '#a9abac',
    '#54585A',
    '#BFB37C',
    '#857437',
]
PALETTE_CMAP = [
    '#66829a',
    # '#003057',
    '#a9abac',
    # '#54585A',
    '#BFB37C',
    # '#857437',
]

PARTICIPANT_CLASS_MAP = {
    'A': 'AtRisk',
    'C': 'Healthy',
    'F': 'AtRisk',
}

#general
import pathlib
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, hex2color
from timeit import default_timer
import gc

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)
plt.rcParams["font.family"] = "Georgia"

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#custom utils module from helpers/utils.py
from helpers.utils import printmd, export_code,format_timing
from helpers.model_funcs import add_labels, create_train_test, score_model, generate_heatmap

#modeling
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier#, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC
from multiprocessing import cpu_count
from xgboost import XGBClassifier

gt_cmap = LinearSegmentedColormap.from_list("custom_cmap", [hex2color(color) for color in PALETTE_CMAP])


## <span style='color:#003057'>Read in Epochs</span>

In [2]:
epoch_file = 'data/EpochData.pkl'
data_file = 'data/ConsolidatedParticipantData.pkl'

with open(data_file, 'rb') as f:
    data_dict = pickle.load(f)

participants = data_dict['participants']
participants['Class'] = participants.Group.map(PARTICIPANT_CLASS_MAP)
del data_dict
gc.collect() # delete rest of data which is not needed

with open(epoch_file, 'rb') as f:
    epochs = pickle.load(f)

printmd('# <b><u>Data Summary</u></b>')
printmd(f'## <u>Participant Data Sample Preview:</u>')
print('Number of participants:', len(participants))
for i in participants.Group.unique():
    print(f'# of participants in Group {i}:', len(participants[participants.Group == i]))
for i in participants.Class.unique():
    print(f'# of participants in Class {i}:', len(participants[participants.Class == i]))
display(participants.sample(5, random_state=SEED_VAL))
participants.info()

printmd(f'## <u>Epoch Data Sample Preview:</u>')
print('Number of epoch datasets:', len(epochs))
print('Epoch dataset keys:')
print('|'.join(list(epochs.keys())))
printmd(f'### <u>Epoch Data (2000ms segments, 25 percent overlap) Sample Preview:</u>')
prev_epoch = epochs['2000_25']
display(prev_epoch.sample(5, random_state=SEED_VAL))


# <span style="color:#B3A369"> <b><u>Data Summary</u></b></span>

## <span style="color:#003057"> <u>Participant Data Sample Preview:</u></span>

Number of participants: 88
# of participants in Group A: 36
# of participants in Group C: 29
# of participants in Group F: 23
# of participants in Class AtRisk: 59
# of participants in Class Healthy: 29


,participant_id,Gender,Age,Group,MMSE,Class
20,sub-021,M,79,A,22,AtRisk
47,sub-048,M,65,C,30,Healthy
23,sub-024,F,69,A,20,AtRisk
38,sub-039,M,70,C,30,Healthy
50,sub-051,F,75,C,30,Healthy


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   participant_id  88 non-null     object
 1   Gender          88 non-null     object
 2   Age             88 non-null     int64 
 3   Group           88 non-null     object
 4   MMSE            88 non-null     int64 
 5   Class           88 non-null     object
dtypes: int64(2), object(4)
memory usage: 4.3+ KB


## <span style="color:#003057"> <u>Epoch Data Sample Preview:</u></span>

Number of epoch datasets: 15
Epoch dataset keys:
2000_25|2000_50|2000_75|3000_25|3000_50|3000_75|4000_25|4000_50|4000_75|5000_25|5000_50|5000_75|6000_25|6000_50|6000_75


### <span style="color:#54585A"> <u>Epoch Data (2000ms segments, 25 percent overlap) Sample Preview:</u></span>

,participant_id,seg_start,seg_end,Fp1_delta_psd,Fp1_delta_rbp,Fp1_theta_psd,Fp1_theta_rbp,Fp1_alpha_psd,Fp1_alpha_rbp,Fp1_beta_psd,Fp1_beta_rbp,Fp1_gamma_psd,Fp1_gamma_rbp,Fp2_delta_psd,Fp2_delta_rbp,Fp2_theta_psd,Fp2_theta_rbp,Fp2_alpha_psd,Fp2_alpha_rbp,Fp2_beta_psd,Fp2_beta_rbp,Fp2_gamma_psd,Fp2_gamma_rbp,F3_delta_psd,F3_delta_rbp,F3_theta_psd,F3_theta_rbp,F3_alpha_psd,F3_alpha_rbp,F3_beta_psd,F3_beta_rbp,F3_gamma_psd,F3_gamma_rbp,F4_delta_psd,F4_delta_rbp,F4_theta_psd,F4_theta_rbp,F4_alpha_psd,F4_alpha_rbp,F4_beta_psd,F4_beta_rbp,F4_gamma_psd,F4_gamma_rbp,C3_delta_psd,C3_delta_rbp,C3_theta_psd,C3_theta_rbp,C3_alpha_psd,C3_alpha_rbp,C3_beta_psd,C3_beta_rbp,C3_gamma_psd,C3_gamma_rbp,C4_delta_psd,C4_delta_rbp,C4_theta_psd,C4_theta_rbp,C4_alpha_psd,C4_alpha_rbp,C4_beta_psd,C4_beta_rbp,C4_gamma_psd,C4_gamma_rbp,P3_delta_psd,P3_delta_rbp,P3_theta_psd,P3_theta_rbp,P3_alpha_psd,P3_alpha_rbp,P3_beta_psd,P3_beta_rbp,P3_gamma_psd,P3_gamma_rbp,P4_delta_psd,P4_delta_rbp,P4_theta_psd,P4_theta_rbp,P4_alpha_psd,P4_alpha_rbp,P4_beta_psd,P4_beta_rbp,P4_gamma_psd,P4_gamma_rbp,O1_delta_psd,O1_delta_rbp,O1_theta_psd,O1_theta_rbp,O1_alpha_psd,O1_alpha_rbp,O1_beta_psd,O1_beta_rbp,O1_gamma_psd,O1_gamma_rbp,O2_delta_psd,O2_delta_rbp,O2_theta_psd,O2_theta_rbp,O2_alpha_psd,O2_alpha_rbp,O2_beta_psd,O2_beta_rbp,O2_gamma_psd,O2_gamma_rbp,F7_delta_psd,F7_delta_rbp,F7_theta_psd,F7_theta_rbp,F7_alpha_psd,F7_alpha_rbp,F7_beta_psd,F7_beta_rbp,F7_gamma_psd,F7_gamma_rbp,F8_delta_psd,F8_delta_rbp,F8_theta_psd,F8_theta_rbp,F8_alpha_psd,F8_alpha_rbp,F8_beta_psd,F8_beta_rbp,F8_gamma_psd,F8_gamma_rbp,T3_delta_psd,T3_delta_rbp,T3_theta_psd,T3_theta_rbp,T3_alpha_psd,T3_alpha_rbp,T3_beta_psd,T3_beta_rbp,T3_gamma_psd,T3_gamma_rbp,T4_delta_psd,T4_delta_rbp,T4_theta_psd,T4_theta_rbp,T4_alpha_psd,T4_alpha_rbp,T4_beta_psd,T4_beta_rbp,T4_gamma_psd,T4_gamma_rbp,T5_delta_psd,T5_delta_rbp,T5_theta_psd,T5_theta_rbp,T5_alpha_psd,T5_alpha_rbp,T5_beta_psd,T5_beta_rbp,T5_gamma_psd,T5_gamma_rbp,T6_delta_psd,T6_delta_rbp,T6_theta_psd,T6_theta_rbp,T6_alpha_psd,T6_alpha_rbp,T6_beta_psd,T6_beta_rbp,T6_gamma_psd,T6_gamma_rbp,Fz_delta_psd,Fz_delta_rbp,Fz_theta_psd,Fz_theta_rbp,Fz_alpha_psd,Fz_alpha_rbp,Fz_beta_psd,Fz_beta_rbp,Fz_gamma_psd,Fz_gamma_rbp,Cz_delta_psd,Cz_delta_rbp,Cz_theta_psd,Cz_theta_rbp,Cz_alpha_psd,Cz_alpha_rbp,Cz_beta_psd,Cz_beta_rbp,Cz_gamma_psd,Cz_gamma_rbp,Pz_delta_psd,Pz_delta_rbp,Pz_theta_psd,Pz_theta_rbp,Pz_alpha_psd,Pz_alpha_rbp,Pz_beta_psd,Pz_beta_rbp,Pz_gamma_psd,Pz_gamma_rbp
29127,sub-054,180000,182000,101.619217,0.674185,24.120581,0.160026,13.992456,0.092832,7.909776,0.052477,3.087018,0.020481,106.206696,0.650375,25.143023,0.153968,15.774080,0.096595,9.856347,0.060357,6.320441,0.038704,94.978004,0.720623,16.586493,0.125846,10.386333,0.078804,6.590670,0.050005,3.258326,0.024722,114.300560,0.725754,19.301350,0.122554,11.996744,0.076174,8.046854,0.051094,3.846699,0.024425,109.046425,0.820300,10.272523,0.077275,6.122187,0.046054,5.373770,0.040424,2.119812,0.015946,105.209030,0.768277,14.820225,0.108223,9.143145,0.066767,5.803803,0.042382,1.965263,0.014351,109.505478,0.726228,12.729460,0.084420,15.929302,0.105641,10.169971,0.067446,2.452374,0.016264,106.124802,0.685430,15.796976,0.102028,21.683306,0.140046,9.526265,0.061527,1.698288,0.010969,117.238678,0.870784,9.980938,0.074133,3.290785,0.024442,2.715827,0.020172,1.409547,0.010469,107.130447,0.496425,35.389694,0.163990,58.785660,0.272403,12.245325,0.056743,2.252955,0.010440,100.778702,0.673897,26.203289,0.175219,12.774203,0.085420,7.034098,0.047036,2.755835,0.018428,110.778961,0.616390,34.058128,0.189504,17.403919,0.096838,12.752608,0.070957,4.728472,0.026310,100.521866,0.664806,26.433134,0.174817,10.504236,0.069470,7.347775,0.048595,6.397784,0.042312,113.974426,0.660282,25.360271,0.146918,15.626774,0.090530,12.071275,0.069932,5.581991,0.032338,106.125084,0.573431,30.856989,0.166731,35.670456,0.192740,8.786214,0.047475,3.631526,0.019622,108.584183,0.481103,30.425066,0.134804,70.929192,0.314265,13.501643,0.059822,

# <span style='color:#B3A369'> Create Basis Models</span>

In [3]:
debug_mode = True

models = {
    'RF': RandomForestClassifier(random_state=SEED_VAL, n_jobs = -1),
    'XGB': XGBClassifier(n_jobs=cpu_count(), random_state=SEED_VAL),
    'LR': LogisticRegression(random_state=SEED_VAL, n_jobs=-1),
    'KNN': KNeighborsClassifier(n_jobs = -1),
    # 'GB': GradientBoostingClassifier(random_state=SEED_VAL),
    # 'SVC': SVC(random_state=SEED_VAL)
    
}

results_dict = {model_name: {} for model_name in models.keys()}  # To store results for each model

s_modeling = default_timer()
for key in epochs.keys():
    epoch_params = key.split('_')
    epoch_params = f'{epoch_params[0]}ms segments with {epoch_params[1]} percent overlap'
    printmd(f'## <u>{epoch_params}</u>')
    
    #! Label Encoding
    data_tmp = epochs[key].copy()
    encoder_group = LabelEncoder() # need to initialize separately to get the report later
    s_labeling = default_timer()
    data_labled = add_labels(data_tmp, participants, encoder_group)
    if debug_mode: print('Labeling Time:', format_timing(default_timer() - s_labeling))
    
    #! Train Test Split
    s_split = default_timer()
    x_train, x_test, y_train, y_test, id_train, id_test= create_train_test(data_labled)
    if debug_mode: print('Split Time:', format_timing(default_timer() - s_split))

    #! Modeling
    for model_name, model in models.items():   
        printmd(f'### {model_name}')
        
        s_fit = default_timer()
        model.fit(x_train, y_train)
        if debug_mode: print('Fit Time:', format_timing(default_timer() - s_fit))
        
        s_predict = default_timer()
        y_pred = model.predict(x_test)
        y_pred_group = encoder_group.inverse_transform(y_pred)
        y_pred_class = np.array([PARTICIPANT_CLASS_MAP[x] for x in y_pred_group])
        if debug_mode: print('Prediction Time:', format_timing(default_timer() - s_predict))
        
        s_scoring = default_timer()
        # results_dict[model_name][key] = score_model(id_test, y_pred_group, participants, encoder_group, False)
        results_dict[model_name][key] = {
            'model': model,
            'encoder': encoder_group,
            'scores': score_model(id_test, y_pred_group, participants, encoder_group, False)
        }
        
        if debug_mode: print('Scoring Time:', format_timing(default_timer() - s_scoring))
        
    printmd('---')

printmd(f'# All models have been trained and scored.')
printmd(f'Total time elapsed: {format_timing(default_timer() - s_modeling)}')

## <span style="color:#003057"> <u>2000ms segments with 25 percent overlap</u></span>

Labeling Time: 0:00:00.039584
Split Time: 0:00:00.077250


### <span style="color:#54585A"> RF</span>

Fit Time: 0:00:03.868958
Prediction Time: 0:00:00.030438
Scoring Time: 0:00:00.017913


### <span style="color:#54585A"> XGB</span>

...

### <span style="color:#54585A"> KNN</span>

Fit Time: 0:00:00.003082
Prediction Time: 0:00:00.669819
Scoring Time: 0:00:00.010832


---

# <span style="color:#B3A369"> All models have been trained and scored.</span>

Total time elapsed: 0:02:28.491505

In [4]:
printmd(f'# <u>Results will be Saved to Pickle</u>')

for k in results_dict.keys():
    printmd (f'## {k}')
    for sk in results_dict[k].keys():
        printmd(f'### {sk}')
        display(results_dict[k][sk]['scores'])

with open('data/BasisModelResults.pkl', 'wb') as f:
    pickle.dump(results_dict, f)
printmd('---')
print('Results saved to data/BasisModelResults.pkl')

# <span style="color:#B3A369"> <u>Results will be Saved to Pickle</u></span>

## <span style="color:#003057"> RF</span>

### <span style="color:#54585A"> 2000_25</span>

{'Predictions':    participant_id    A    C   F predicted_group true_group predicted_class  \
 0         sub-001   78    1   1               A          A          AtRisk   
 1         sub-002   79   21   0               A          A          AtRisk   
 2         sub-003   28   11   0               A          A          AtRisk   
 3         sub-004  106    0   4               A          A          AtRisk   
 4         sub-005   78   11  11               A          A          AtRisk   
 5         sub-006   71   14   2               A          A          AtRisk   
 6         sub-007  106    3   0               A          A          AtRisk   
 7         sub-008  107    1   1               A          A          AtRisk   
 8         sub-009   60   19   3               A          A          AtRisk   
 9         sub-010  122   47   7               A          A          AtRisk   
 10        sub-011   89   20  10               A          A          AtRisk   
 11        sub-012  100   15   5     

### <span style="color:#54585A"> 2000_50</span>

...

{'Predictions':    participant_id   A   C   F predicted_group true_group predicted_class  \
 0         sub-001  41   1   1               A          A          AtRisk   
 1         sub-002  40  11   7               A          A          AtRisk   
 2         sub-003  16   3   0               A          A          AtRisk   
 3         sub-004  43   0   2               A          A          AtRisk   
 4         sub-005  46   7   5               A          A          AtRisk   
 5         sub-006  34   1   0               A          A          AtRisk   
 6         sub-007  41   1   6               A          A          AtRisk   
 7         sub-008  51   2   3               A          A          AtRisk   
 8         sub-009  41   3   1               A          A          AtRisk   
 9         sub-010  77  13   1               A          A          AtRisk   
 10        sub-011  42   5   3               A          A          AtRisk   
 11        sub-012  40  11   8               A          A    

### <span style="color:#54585A"> 6000_75</span>

{'Predictions':    participant_id    A    C    F predicted_group true_group predicted_class  \
 0         sub-001   74    1    5               A          A          AtRisk   
 1         sub-002   87    5    8               A          A          AtRisk   
 2         sub-003   36    2    1               A          A          AtRisk   
 3         sub-004  110    0    0               A          A          AtRisk   
 4         sub-005   84    8    8               A          A          AtRisk   
 5         sub-006   85    1    1               A          A          AtRisk   
 6         sub-007  100    0    9               A          A          AtRisk   
 7         sub-008   98    0   11               A          A          AtRisk   
 8         sub-009   81    1    0               A          A          AtRisk   
 9         sub-010  157   15    4               A          A          AtRisk   
 10        sub-011  114    3    2               A          A          AtRisk   
 11        sub-012   84  

---

Results saved to data/BasisModelResults.pkl


# <span style='color:#B3A369'> Export Source Code </span>

In [5]:
user = str(pathlib.Path.home()).split('\\')[-1]
export_flag = False
if user != 'bkeith' and user != 'Brian':
    raise Exception('User running code is not the student. No need to run below.')
else:
    export_flag = True
    print('User running code is the student. Continue to file Export.')

cur_file = __vsc_ipynb_file__
output_dir = 'zlogs'
output_name = cur_file.split('\\')[-1].split('.')[0] + f'.html'
cell_tags_exist = False
template = 'classic'

if export_flag == True:
    export_code(cur_file,output_dir, output_name, cell_tags_exist, template=template)

User running code is the student. Continue to file Export.


### <span style="color:#54585A"> Code saved to 03 Generate Basis Models.html</span>

Replacing GeorgiaTech_RGB.png
